# Assignment 4
* Unit: SIT720
* Name: Bryon Baker
* Student #: 85031775

In [1]:
#!pip3 install pandas
#!pip3 install numpy

In [2]:
import sys
import warnings
import random
import pandas as pd # dataframe manipulation
import numpy as np # linear algebra

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

In [3]:
# Seed the random nunmber generator to ensure K-Means randomisation works properly.
random.seed()

print(f"Seeding random number generator. Random number is: {random.random()}")

Seeding random number generator. Random number is: 0.7214874833471027


# Helper Functions
This section defines any helper functions used in the code below

In [4]:
# Helper function to take the inverse ln(x+1) of a np.array and return an np.array
def inverse_transform():
    return

In [5]:
# A helper function to standardise all the values in the dataset with a mean=0 and SD=1
def standardise_dataset( dataset_name, dataset, column_headings ):
    # Standardise all attributes to zero mean and SD 1
    scaler = StandardScaler(copy=False, with_mean=True, with_std=True)
    scaler.fit(dataset)
    result = scaler.transform(dataset)
    # print(f"Transformed Data Frame:\n{SVM_STFWI_df}")
    m = np.mean(result, axis=0)
    s = np.std(result, axis=0)
    #print("Checking transformation")
    #print(f"Column  means:\n{m}")
    #print(f"Column  SD:\n{s}")

    # The StandardScaler converts to an array. Convert back to a DataFrame
    result_df = pd.DataFrame(result)
    result_df.columns = column_headings     # Reassign the column names.
    # print(result_df.head())

    save_dataset( dataset_name, result_df)      # Save the dataset to a file for exploration later
    
    return result_df

In [6]:
from pathlib import Path 
    
def save_dataset( filename, dataset ):
    filename = "~/datasets/SIT720/Ass4/transformed/" + filename + ".csv"
    filepath = Path(filename)  
    filepath.parent.mkdir(parents=True, exist_ok=True)
    dataset.to_csv(filepath)

# Load the source dataset
Dataset is stored in public git repo. COnfigure the ssl context in order to access github raw data files.

In [7]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

url = 'https://raw.githubusercontent.com/bryonbaker/datasets/main/SIT720/Ass4/forestfires.csv'
full_df = pd.read_csv(url)
print(f"{full_df.head()}\n")

   X  Y month  day  FFMC   DMC     DC  ISI  temp  RH  wind  rain  area
0  7  5   mar  fri  86.2  26.2   94.3  5.1   8.2  51   6.7   0.0   0.0
1  7  4   oct  tue  90.6  35.4  669.1  6.7  18.0  33   0.9   0.0   0.0
2  7  4   oct  sat  90.6  43.7  686.9  6.7  14.6  33   1.3   0.0   0.0
3  8  6   mar  fri  91.7  33.3   77.5  9.0   8.3  97   4.0   0.2   0.0
4  8  6   mar  sun  89.3  51.3  102.2  9.6  11.4  99   1.8   0.0   0.0



# Pre Process the Data
1. One hot encode the day and month attributes.
2. ...

In [8]:
# Check the dataset has no NaNs.
# TODO: Check the dtaa quality

if full_df.isnull().sum().sum() == 0:
    print("Data does not have any missing values.")
else:
    print("Data has missing values and needs pre processing.")
    print("Stopping....")
    quit() 

Data does not have any missing values.


In [9]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore', drop=None, dtype='int')

In [10]:
# One-hot encode month
transformed = ohe.fit_transform(full_df[['month']])
ohe_month_df = pd.DataFrame(transformed.toarray())
ohe_month_df.columns = ohe.categories_[0]

In [11]:
# One hot encode day
transformed = ohe.fit_transform(full_df[['day']])
ohe_day_df = pd.DataFrame(transformed.toarray())
ohe_day_df.columns = ohe.categories_[0]

In [12]:
encoded_df = full_df.drop(['month','day'], axis=1)
encoded_df = pd.concat([encoded_df, ohe_month_df], axis=1)
encoded_df = pd.concat([encoded_df, ohe_day_df], axis=1)
encoded_df.head(20)

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,nov,oct,sep,fri,mon,sat,sun,thu,tue,wed
0,7,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,1,0,0,0,0,0,0
1,7,4,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,1,0,0,0,0,0,0,1,0
2,7,4,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,1,0,0,0,1,0,0,0,0
3,8,6,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,1,0,0,0,0,0,0
4,8,6,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,0,0,1,0,0,0
5,8,6,92.3,85.3,488.0,14.7,22.2,29,5.4,0.0,...,0,0,0,0,0,0,1,0,0,0
6,8,6,92.3,88.9,495.6,8.5,24.1,27,3.1,0.0,...,0,0,0,0,1,0,0,0,0,0
7,8,6,91.5,145.4,608.2,10.7,8.0,86,2.2,0.0,...,0,0,0,0,1,0,0,0,0,0
8,8,6,91.0,129.5,692.6,7.0,13.1,63,5.4,0.0,...,0,0,1,0,0,0,0,0,1,0
9,7,5,92.5,88.0,698.6,7.1,22.8,40,4.0,0.0,...,0,0,1,0,0,1,0,0,0,0


In [13]:
# Ordinal encode month
from sklearn.preprocessing import OrdinalEncoder

ore = OrdinalEncoder(dtype='int')

transformed = ore.fit_transform(full_df[['month']])
#print(transformed)
ord_month_df = pd.DataFrame(transformed)
ord_month_df.columns = ['month']

In [14]:
# Ordinal encode month
from sklearn.preprocessing import OrdinalEncoder

ore = OrdinalEncoder(dtype='int')

transformed = ore.fit_transform(full_df[['day']])
#print(transformed)
ord_day_df = pd.DataFrame(transformed)
ord_day_df.columns = ['day']

In [15]:
unencoded_df = full_df     # RF and DT need the unencoded dataframe
unencoded_df = full_df.drop(['month','day'], axis=1)
unencoded_df = pd.concat([unencoded_df, ord_month_df], axis=1)
unencoded_df = pd.concat([unencoded_df, ord_day_df], axis=1)
unencoded_df.head(20)

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,month,day
0,7,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,7,0
1,7,4,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,10,5
2,7,4,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,10,2
3,8,6,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,7,0
4,8,6,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,7,3
5,8,6,92.3,85.3,488.0,14.7,22.2,29,5.4,0.0,0.0,1,3
6,8,6,92.3,88.9,495.6,8.5,24.1,27,3.1,0.0,0.0,1,1
7,8,6,91.5,145.4,608.2,10.7,8.0,86,2.2,0.0,0.0,1,1
8,8,6,91.0,129.5,692.6,7.0,13.1,63,5.4,0.0,0.0,11,5
9,7,5,92.5,88.0,698.6,7.1,22.8,40,4.0,0.0,0.0,11,2


In [16]:
del full_df         # Make sure we don't accidentally use full_df later

## Preprocess area
Reduce the skewness of area as per procedure on page 5.
This will be used to train the model, but it is important to take the inverse log after prediction so as to find the predicted area. 


In [17]:
encoded_df["xfrmd_area"] = encoded_df["area"] +1                 # Create a new area column that will contain the ln(area+1)
print(encoded_df[["area","xfrmd_area"]])
encoded_df["xfrmd_area"] = np.log(encoded_df['xfrmd_area'])      # Take the ln(area+1)
print(encoded_df[["xfrmd_area"]])                                # Have a look and make sure it worked....

      area  xfrmd_area
0     0.00        1.00
1     0.00        1.00
2     0.00        1.00
3     0.00        1.00
4     0.00        1.00
..     ...         ...
512   6.44        7.44
513  54.29       55.29
514  11.16       12.16
515   0.00        1.00
516   0.00        1.00

[517 rows x 2 columns]
     xfrmd_area
0      0.000000
1      0.000000
2      0.000000
3      0.000000
4      0.000000
..          ...
512    2.006871
513    4.012592
514    2.498152
515    0.000000
516    0.000000

[517 rows x 1 columns]


## Create Sub-Datasets

Now we have all the raw ingredients for all datasets, create the sub datasets that the different models will need.

In [18]:
# Define the sub datasets from the transformed dataset

# Get the 1-of-C encoded column names for month and day
month_cols = ohe_month_df.columns.tolist()
day_cols = ohe_day_df.columns.tolist()

# Construct the dataframe column names
STFWI_encoded_cols = ['X','Y','FFMC','DMC','DC','ISI']
STFWI_encoded_cols = STFWI_encoded_cols + month_cols + day_cols
STM_encoded_cols = ['X','Y','temp','RH','wind','rain']
STM_encoded_cols = STM_encoded_cols + month_cols + day_cols
FWI_encoded_cols = ['FFMC','DMC','DC','ISI']
M_encoded_cols = ['temp','RH','wind','rain']

# Construct the dataframe column names
STFWI_unencoded_cols = ['X','Y','month','day','FFMC','DMC','DC','ISI']
STM_unencoded_cols = ['X','Y','month','day','temp','RH','wind','rain']
FWI_unencoded_cols = ['FFMC','DMC','DC','ISI']
M_unencoded_cols = ['temp','RH','wind','rain']


In [19]:
# create feature tables for MR, SVM, and NN
STFWI_encoded_df = encoded_df[STFWI_encoded_cols]
STM_encoded_df = encoded_df[STM_encoded_cols]
FWI_encoded_df = encoded_df[FWI_encoded_cols]
M_encoded_df = encoded_df[M_encoded_cols]
# Create the feature tables for RF and DT
STFWI_unencoded_df = unencoded_df[STFWI_unencoded_cols]
STM_unencoded_df = unencoded_df[STM_unencoded_cols]
FWI_unencoded_df = unencoded_df[FWI_unencoded_cols]
M_unencoded_df = unencoded_df[M_unencoded_cols]
# Create the transformed area ln(area+1)
y_xformed = encoded_df["xfrmd_area"]

In [20]:
print("SVM, MR, and NN Dataset columns:")
print("="*16)
print(f"STFWI Dataset columns:\n{STFWI_encoded_df.columns}\n")
print(f"STM Dataset columns:\n{STM_encoded_df.columns}\n")
print(f"FWI Dataset columns:\n{FWI_encoded_df.columns}\n")
print(f"M Dataset columns:\n{M_encoded_df.columns}\n")
print("="*16)
print("DT and RF Dataset columns:")
print("="*16)
print(f"STFWI Dataset columns:\n{STFWI_unencoded_df.columns}\n")
print(f"STM Dataset columns:\n{STM_unencoded_df.columns}\n")
print(f"FWI Dataset columns:\n{FWI_unencoded_df.columns}\n")
print(f"M Dataset columns:\n{M_unencoded_df.columns}\n")

SVM, MR, and NN Dataset columns:
STFWI Dataset columns:
Index(['X', 'Y', 'FFMC', 'DMC', 'DC', 'ISI', 'apr', 'aug', 'dec', 'feb', 'jan',
       'jul', 'jun', 'mar', 'may', 'nov', 'oct', 'sep', 'fri', 'mon', 'sat',
       'sun', 'thu', 'tue', 'wed'],
      dtype='object')

STM Dataset columns:
Index(['X', 'Y', 'temp', 'RH', 'wind', 'rain', 'apr', 'aug', 'dec', 'feb',
       'jan', 'jul', 'jun', 'mar', 'may', 'nov', 'oct', 'sep', 'fri', 'mon',
       'sat', 'sun', 'thu', 'tue', 'wed'],
      dtype='object')

FWI Dataset columns:
Index(['FFMC', 'DMC', 'DC', 'ISI'], dtype='object')

M Dataset columns:
Index(['temp', 'RH', 'wind', 'rain'], dtype='object')

DT and RF Dataset columns:
STFWI Dataset columns:
Index(['X', 'Y', 'month', 'day', 'FFMC', 'DMC', 'DC', 'ISI'], dtype='object')

STM Dataset columns:
Index(['X', 'Y', 'month', 'day', 'temp', 'RH', 'wind', 'rain'], dtype='object')

FWI Dataset columns:
Index(['FFMC', 'DMC', 'DC', 'ISI'], dtype='object')

M Dataset columns:
Index(['temp', 'R

## Decision Tree Dataset Preparation

In [21]:
DT_STFWI_df = STFWI_unencoded_df
DT_STM_df = STM_unencoded_df
DT_FWI_df = FWI_unencoded_df
DT_M_df = M_unencoded_df

save_dataset( "DT_STFWI_df", DT_STFWI_df )
save_dataset( "DT_STM_df", DT_STM_df )
save_dataset( "DT_FWI_df", DT_FWI_df )
save_dataset( "DT_M_df", DT_M_df )



## Random Forrest Dataset Preparation

In [22]:
RF_STFWI_df = STFWI_unencoded_df
RF_STM_df = STM_unencoded_df
RF_FWI_df = FWI_unencoded_df
RF_M_df = M_unencoded_df

save_dataset( "RF_STFWI_df", RF_STFWI_df )
save_dataset( "RF_STM_df", RF_STM_df )
save_dataset( "RF_FWI_df", RF_FWI_df )
save_dataset( "RF_M_df", RF_M_df )

## SVM Data Set Preparation

In [23]:
# Standardise all attributes to zero mean and SD 1
SVM_STFWI_df = STFWI_encoded_df
SVM_STM_df = STM_encoded_df
SVM_FWI_df = FWI_encoded_df
SVM_M_df = M_encoded_df

SVM_STFWI_df = standardise_dataset("SVM_STFWI_df", SVM_STFWI_df, STFWI_encoded_cols)
SVM_STM_df = standardise_dataset("SVM_STM_df", SVM_STM_df, STM_encoded_cols)
SVM_FWI_df = standardise_dataset("SVM_FWI_df", SVM_FWI_df, FWI_encoded_cols)
SVM_M_df = standardise_dataset("SVM_M_df", SVM_M_df, M_encoded_cols)

## Multiple Rgression Data Set Preparation

In [24]:
MR_STFWI_df = STFWI_encoded_df
MR_STM_df = STM_encoded_df
MR_FWI_df = FWI_encoded_df
MR_M_df = M_encoded_df

MR_STFWI_df = standardise_dataset("MR_STFWI_df", MR_STFWI_df, STFWI_encoded_cols)
MR_STM_df = standardise_dataset("MR_STM_df", MR_STM_df, STM_encoded_cols)
MR_FWI_df = standardise_dataset("MR_FWI_df", MR_FWI_df, FWI_encoded_cols)
MR_M_df = standardise_dataset("MR_M_df", MR_M_df, M_encoded_cols)

## Neural Network Dataset Preparation



In [25]:
NN_STFWI_df = STFWI_encoded_df
NN_STM_df = STM_encoded_df
NN_FWI_df = FWI_encoded_df
NN_M_df = M_encoded_df

NN_STFWI_df = standardise_dataset("NN_STFWI_df", NN_STFWI_df, STFWI_encoded_cols)
NN_STM_df = standardise_dataset("NN_STM_df", NN_STM_df, STM_encoded_cols)
NN_FWI_df = standardise_dataset("NN_FWI_df", NN_FWI_df, FWI_encoded_cols)
NN_M_df = standardise_dataset("NN_M_df", NN_M_df, M_encoded_cols)

# Define All Hyperparameters
Define all the hyperparameters from the paper that will be used in the models.

There hyperparameters are declared in a single location to aid in marking.

In [26]:
# Multiple Regression
fit_intercept = True
copy_X = True
n_jobs = None
positive = False

# Descision Tree
min_samples_split = 2

# Random Forrest
n_trees = 500

# SVM

# Neural Network (HD Task)


# Create the Results DataFrame

In [27]:
results_df = pd.DataFrame(columns=['Model','STFWI (RMSE)','STM (RMSE)','FWI (RMSE)','M (RMSE)'])

# Calculate the Naive results

In [28]:
# Generate an array with the predicted values being the mean of the area
y = encoded_df["area"]
y_pred = np.mean(y)
y_naive_pred = np.repeat(y_pred, len(y))

# Calculate the naive RMSE
naive_rmse = mean_squared_error(y, y_naive_pred, squared=False)
print(naive_rmse)

# Add the naive results to the Results Table
rmse_df = pd.DataFrame([['Naive', naive_rmse, naive_rmse, naive_rmse, naive_rmse]], columns=['Model','STFWI (RMSE)','STM (RMSE)','FWI (RMSE)','M (RMSE)'])
results_df.append(rmse_df)

63.59422598281833


,Model,STFWI (RMSE),STM (RMSE),FWI (RMSE),M (RMSE)
0,Naive,63.594226,63.594226,63.594226,63.594226


# Decision Tree

Scoring details:

RMSE: https://scikit-learn.org/stable/modules/model_evaluation.html

https://stackoverflow.com/questions/62514395/score-obtained-from-cross-val-score-is-rmse-or-mse


In [29]:
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

print("DECISION TREE")
print("*"*30)

# Get the list of training / validating datasets
train_set = { "STFWI" : DT_STFWI_df, "STM" : DT_STM_df, "FWI" : DT_FWI_df, "M" : DT_M_df}

le = LabelEncoder()
le.fit(y_xformed)
y = le.transform(y_xformed)

for key in train_set:
    print(key)
    print("="*10)

    X = train_set[key]

    model = DecisionTreeClassifier()
    accuracies = cross_val_score(estimator = model, X=X,y=y,scoring='neg_mean_squared_error', cv =10)
    print(accuracies.mean()*-1)
    
del train_set, y, le

DECISION TREE
******************************
STFWI
13060.216553544495
STM


/opt/app-root/lib64/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
/opt/app-root/lib64/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


13260.842383107089
FWI
11022.58250377074
M
11632.376734539972


/opt/app-root/lib64/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
/opt/app-root/lib64/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


In [30]:
from sklearn.tree import DecisionTreeRegressor


print("DECISION TREE REGRESSOR")
print("*"*30)


# Get the list of training / validating datasets
train_set = { "STFWI" : DT_STFWI_df, "STM" : DT_STM_df, "FWI" : DT_FWI_df, "M" : DT_M_df}

y = y_xformed

for key in train_set:
    print(key)
    print("="*10)
    
    model = DecisionTreeRegressor()
    X = train_set[key]
    accuracies = cross_val_score(estimator = model, X=X,y=y,scoring='neg_mean_squared_error', cv =10)
    print(accuracies)
    print(accuracies.mean()*-1)

del train_set, y     # Delete the variables you do not want to be picked up accidentally elsewhere.

DECISION TREE REGRESSOR
******************************
STFWI
[-5.0762973  -4.17653017 -2.27406164 -3.63362508 -8.73516238 -5.60856937
 -2.47486419 -5.36696597 -5.43001798 -4.49751763]
4.727361173535554
STM
[-4.84678381 -9.94882337 -3.12615627 -2.70970571 -8.7239473  -3.88167535
 -4.23334426 -5.66773202 -4.1786138  -4.78419617]
5.210097806196996
FWI
[-2.67378843 -2.72275649 -2.5148074  -2.55429274 -8.65678947 -3.9104512
 -1.93694721 -3.15967647 -5.47236944 -3.60722464]
3.72091034891031
M
[-3.07470742 -2.58476155 -2.79254194 -4.39997962 -8.77532067 -2.47414123
 -3.49837019 -5.34983638 -2.99314217 -3.73968235]
3.968248352308754


# Multiple Regression

In [31]:
from sklearn.linear_model import LinearRegression

print("LINEAR REGRESSION")
print("*"*30)

# Get the list of training / validating datasets
train_set = { "STFWI" : MR_STFWI_df, "STM" : MR_STM_df, "FWI" : MR_FWI_df, "M" : MR_M_df}

y = y_xformed
for key in train_set:
    print(key)
    print("="*10)
    
    X = train_set[key]
    model = LinearRegression()
    accuracies = cross_val_score(estimator = model, X=X,y=y,scoring='neg_mean_squared_error', cv =10)
    print(accuracies)
    print(accuracies.mean()*-1)
    
del train_set, y     # Delete the variables you do not want to be picked up accidentally elsewhere.

LINEAR REGRESSION
******************************
STFWI
[-2.52226181 -1.68139779 -1.20623764 -1.38250982 -7.07447268 -2.1642208
 -1.19202203 -2.24853263 -2.24452809 -3.2546044 ]
2.4970787707286504
STM
[-2.43181332e+00 -1.50412358e+00 -1.15254501e+00 -1.39927372e+00
 -7.01583495e+00 -1.24635942e+25 -1.23252054e+00 -2.34186918e+00
 -2.03792498e+00 -2.29313346e+25]
3.5394928810156664e+24
FWI
[-1.59755408 -1.48798314 -1.06899254 -1.1670803  -6.46472    -1.51084945
 -1.11641422 -2.2826806  -2.14442706 -2.71589789]
2.155659929244849
M
[-1.51541083 -1.60587694 -1.12455654 -1.1694122  -6.515222   -1.63634864
 -1.10354378 -2.46564148 -1.94648773 -4.54220426]
2.3624704414477065
